In [3]:
import psycopg2
import requests
import json
import time
from psycopg2.extras import Json
import pandas as pd
import numpy as np

In [4]:
conn = psycopg2.connect("dbname=postgres user=postgres host=localhost port=5432 password=123456")

In [5]:
base_url = 'https://api.hh.ru/vacancies'
params = {
    "text": "NAME:Аналитик",
    # "search_field":[ "exprerience", "schedule"], 
    "exprerience":"noExperience",
    "schedule": 'remote',
    "period": 3,
    "per_page": 100
}

In [6]:
r = requests.get(base_url, params=params)

In [7]:
j = json.loads(r.content)

In [8]:
j['found']

942

In [9]:
j['items'][4]

{'id': '106100439',
 'premium': False,
 'name': 'Менеджер – аналитик маркетплейса OZON',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '2',
  'name': 'Санкт-Петербург',
  'url': 'https://api.hh.ru/areas/2'},
 'salary': {'from': 100000, 'to': None, 'currency': 'RUR', 'gross': False},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': None,
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2024-09-05T10:03:05+0300',
 'created_at': '2024-09-05T10:03:05+0300',
 'archived': False,
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=106100439',
 'insider_interview': None,
 'url': 'https://api.hh.ru/vacancies/106100439?host=hh.ru',
 'alternate_url': 'https://hh.ru/vacancy/106100439',
 'relations': [],
 'employer': {'id': '4438425',
  'name': 'Михайлова Анастасия Алексеевна',
  'url': 'https://api.hh.ru/employers/4438425',
  'alternate_url': 'https://hh.ru/employer/4438425',
  'logo_urls': None,
 

In [11]:
vac = j['items'][1]

In [12]:
vac['schedule']

{'id': 'remote', 'name': 'Удаленная работа'}

In [13]:
conn.commit()

In [14]:
def insert_vacancy(vac):
    cur = conn.cursor()
    cur.execute("insert into analyst_2 values (%s, %s, %s, %s, %s, %s)",
                (vac['id'],
                 vac['name'],     
                 (lambda area: area['name'] if area else None)(vac['area']),
                 (lambda salary: salary['from'] if salary else None)(vac['salary']),
                 (lambda salary: salary['to'] if salary else None)(vac['salary']),
                 (lambda schedule: schedule['currency'] if schedule else None)(vac['salary'])))
    conn.commit()
    cur.close()

In [15]:
def save_vacancy(page=0):
    print('Downloading page', page)
    r = requests.get(base_url, params=dict(params, page=page))
    time.sleep(1)
    result = json.loads(r.content)
    for v in result['items']:
        insert_vacancy(v)
    if len(result['items']) == 0:
        return
    else:
        save_vacancy(page + 1)

In [16]:
save_vacancy()

In [66]:
# cur = conn.cursor()
# cur.execute("select id, area, name, salary_from, salary_to, currency from analyst_2;")
# vacs = cur.fetchall()

In [67]:
# Данные записаны в базу, чтобы получить новые очистите базу

In [68]:
#######################